<a href="https://colab.research.google.com/github/tanakakeitaro/r_d/blob/master/seq2seq_attention_575.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My Drive/'俳句&川柳データセット'/marusen575

/content/drive/My Drive/俳句&川柳データセット/marusen575


In [0]:
from chainer import Chain, Variable, links, optimizers, optimizer, functions, serializers
import chainer
import numpy as np
import pickle
import time
import random
# データセットを分割するモジュールの読み込み
from sklearn.model_selection import train_test_split

In [0]:
W = np.array([
              [0, 0, 0],
              [1, 1, 1],
              [2, 2, 2]]).astype(np.float32)
l = links.EmbedID(W.shape[0], W.shape[1], initialW=W)

In [0]:
l = links.EmbedID(10, 50, ignore_label=1)

In [0]:
x = np.array([2, 5, 1, 3]).astype(np.int32)
y = l(x)
y

variable([[-0.72052014, -0.05138012, -0.27986604, -1.2555176 ,
           -0.93823355, -0.9335817 ,  1.4578001 , -0.533532  ,
           -0.16433288,  0.44913423, -0.4262043 , -0.0037776 ,
           -0.31365454,  0.75423056, -1.6475728 ,  1.5726223 ,
           -0.20037477, -0.5077814 ,  1.5955125 , -0.7155665 ,
           -0.8140466 ,  0.50452685,  1.0830566 , -0.9062343 ,
           -0.20746495,  1.8931264 ,  0.11392874,  1.1904873 ,
           -0.8993586 , -0.41180018,  2.8627884 , -1.5355136 ,
            1.5619587 , -0.45880258, -0.5258363 ,  0.6606859 ,
            2.3301973 , -1.596562  ,  1.1451398 , -0.36461523,
           -0.16047572, -1.3636183 , -0.73220277, -0.25376236,
           -0.42955297, -2.1224468 , -0.28091738, -0.40797278,
            0.746597  ,  0.8466254 ],
          [-0.84835565,  1.5131326 ,  0.22177607, -0.5709425 ,
            1.3266495 , -0.4504019 ,  0.0102353 , -0.04598566,
           -0.9511705 ,  0.4194501 ,  0.3332186 ,  0.11773057,
            1.083

In [0]:
y.shape

(4, 50)

In [0]:
def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data
kami5_data = np.array(pickle_load('marusen_kami5_list.pickle'))
naka7_data = np.array(pickle_load('marusen_naka7_list.pickle'))

In [0]:
id_to_word = pickle_load('id_to_word_marusen575.pickle')
word_to_id = pickle_load('word_to_id_marusen575.pickle')
print(len(id_to_word), len(word_to_id))

20334 20334


In [0]:
# naka7リストの末尾に<eos>を追加
[l.append(1) for l in naka7_data]

In [0]:
# reversedリストの作成
kami5_reversed_data = [list(reversed(l)) for l in kami5_data]

In [0]:
# enc_wordsとdec_wordsの準備
data = np.array([[i, j] for i, j in zip(kami5_data, naka7_data)])
data

array([[list([3, 4]), list([5, 6, 7, 1])],
       [list([9, 10]), list([11, 12, 13, 1])],
       [list([15, 13]), list([16, 17, 18, 1])],
       ...,
       [list([98]), list([76, 18, 1327, 4, 1])],
       [list([263, 18]), list([4140, 39, 1334, 4, 1])],
       [list([5618]), list([20332, 20333, 41, 1])]], dtype=object)

In [0]:
x_train, x_test = train_test_split(data, test_size=0.2, random_state=0)
print(x_train, x_train.shape)

[[list([251, 13, 5033]) list([79, 33, 295, 18, 1])]
 [list([888, 1280, 18]) list([1281, 1282, 1])]
 [list([14941, 13]) list([1200, 41, 19834, 43, 1])]
 ...
 [list([8499, 13]) list([639, 7, 10081, 1])]
 [list([12338]) list([14725, 41, 1])]
 [list([4332]) list([2377, 33, 799, 4, 1])]] (14654, 2)


In [0]:
class LSTM_Encoder(Chain):
    def __init__(self, vocab_size, embed_size, hidden_size):
        """
        クラスの初期化
        :param vocab_size: 使われる単語の種類数（語彙数）
        :param embed_size: 単語をベクトル表現した際のサイズ
        :param hidden_size: 中間層のサイズ
        """
        super(LSTM_Encoder, self).__init__(
            # 単語を単語ベクトルに変換する層
            xe = links.EmbedID(vocab_size, embed_size, ignore_label=-1),
            # 単語ベクトルを隠れ層の4倍のサイズのベクトルに変換する層
            eh = links.Linear(embed_size, 4 * hidden_size),
            # 出力された中間層を4倍のサイズに変換するための層
            hh = links.Linear(hidden_size, 4 * hidden_size)
        )

    def __call__(self, x, c, h):
        """
        Encoderの計算
        :param x: one-hotなベクトル
        :param c: 内部メモリ
        :param h: 隠れ層
        :return: 次の内部メモリ、次の隠れ層
        """
        # xeで単語ベクトルに変換して、そのベクトルをtanhで活性化
        e = functions.tanh(self.xe(x))
        # 前の内部メモリの値cと単語ベクトルの4倍サイズ+中間層の4倍サイズを入力
        return functions.lstm(c, self.eh(e) + self.hh(h))

In [0]:
class Att_LSTM_Decoder(Chain):
    def __init__(self, vocab_size, embed_size, hidden_size):
        """
        Attention ModelのためのDecoderのインスタンス化
        :param vocab_size: 語彙数
        :param embed_size: 単語ベクトルのサイズ
        :param hidden_size: 隠れ層のサイズ
        """
        super(Att_LSTM_Decoder, self).__init__(
            # 単語を単語ベクトルに変換する層
            ye = links.EmbedID(vocab_size, embed_size, ignore_label = -1),
            # 単語ベクトルを隠れ層の4倍のサイズのベクトルに変換する層
            eh = links.Linear(embed_size, 4 * hidden_size),
            # Decoderの中間ベクトルを隠れ層の4倍のサイズのベクトルに変換する層
            hh = links.Linear(hidden_size, 4 * hidden_size),
            # 順向きEncoderの中間ベクトルの加重平均を隠れ層の4倍のサイズのベクトルに変換する層
            fh = links.Linear(hidden_size, 4 * hidden_size),
            # 順向きEncoderの中間ベクトルの加重平均を隠れ層の4倍のサイズのベクトルに変換する層
            bh = links.Linear(hidden_size, 4 * hidden_size),
            # 隠れ層サイズのベクトルを単語ベクトルのサイズに変換する層
            he = links.Linear(hidden_size, embed_size),
            # 単語ベクトルを語彙数サイズのベクトルに変換する層
            ey = links.Linear(embed_size, vocab_size)
        )
    
    def __call__(self, y, c, h, f, b):
        """
        Decoderの計算
        :param y: Decoderに入力する単語
        :param c: 内部メモリ
        :param h: Decoderの中間ベクトル
        :param f: Attention Modelで計算された順向きEncoderの加重平均
        :param b: Attention Modelで計算された逆向きEncoderの加重平均
        :return: 語彙数サイズのベクトル、更新された内部メモリ、更新された中間ベクトル
        """
        # 単語を単語ベクトルに変換
        e = functions.tanh(self.ye(y))
        # 単語ベクトル、Decoderの中間ベクトル、順向きEncoderのAttention、逆向きEncoderのAttentionを使ってLSTM
        c, h = functions.lstm(c, self.eh(e) + self.hh(h) + self.fh(f) + self.bh(b))
        # LSTMから出力された中間ベクトルを語彙数サイズのベクトルに変換する
        t = self.ey(functions.tanh(self.he(h)))
        return t, c, h

In [0]:
class Attention(Chain):
    def __init__(self, hidden_size, flag_gpu):
          """
          Attentionのインスタンス化
          :param hidden_size: 隠れ層のサイズ
          :param flag_gpu: GPUを使うかどうか
          """
          super(Attention, self).__init__(
              # 順向きのEncoderの中間ベクトルを隠れ層サイズのベクトルに変換する線形結合層
              fh = links.Linear(hidden_size, hidden_size),
              # 逆向きのEncoderの中間ベクトルを隠れ層サイズのベクトルに変換する線形結合層
              bh = links.Linear(hidden_size, hidden_size),
              # Decoderの中間ベクトルを隠れ層サイズのベクトルに変換する線形結合層
              hh = links.Linear(hidden_size, hidden_size),
              # 隠れ層サイズのベクトルをスカラーに変換するための線形結合層
              hw = links.Linear(hidden_size, 1)
              )
          # 隠れ層のサイズを記憶
          self.hidden_size = hidden_size
          # GPUを使う場合はcupyを使わないときはnumpyを使う
          if flag_gpu:
              self.ARR = cuda.cupy
          else:
              self.ARR = np

    def __call__(self, fs, bs, h):
          """
          Attentionの計算
          :param fs: 順向きのEncoderの中間ベクトルが記録されたリスト
          :param bs: 逆向きのEncoderの中間ベクトルが記録されたリスト
          :param h: Decoderで出力された中間ベクトル
          :return: N
          """
          #ミニバッチのサイズを記憶
          batch_size = h.data.shape[0]
          # ウェイトを記録するためのリストの初期化
          ws = []
          # ウェイトの合計値を計算するための値を初期化
          sum_w = Variable(self.ARR.zeros((batch_size, 1), dtype='float32'))
          # Encoderの中間ベクトルとDecoderの中間ベクトルを使ってウェイトの計算
          for f, b in zip(fs, bs):
              # 順向きEncoderの中間ベクトル、逆向きEncoderの中間ベクトル、Decoderの中間ベクトルを使ってウェイトの計算
              w = functions.tanh(self.fh(f) + self.bh(b) + self.hh(h))
              # softmax関数を使って正規化する
              w = functions.exp(self.hw(w))
              # 計算したウェイトを記録
              ws.append(w)
              sum_w += w
          # 出力する加重平均ベクトルの初期化
          att_f = Variable(self.ARR.zeros((batch_size, self.hidden_size), dtype = 'float32'))
          att_b = Variable(self.ARR.zeros((batch_size, self.hidden_size), dtype = 'float32'))
          for f, b, w in zip (fs, bs, ws):
              # ウェイトの和が1になるように正規化
              w /= sum_w
              # ウェイト * Encoderの中間ベクトルを出力するベクトルに足していく
              att_f += functions.reshape(functions.batch_matmul(f, w), (batch_size, self.hidden_size))
              att_b += functions.reshape(functions.batch_matmul(b, w), (batch_size, self.hidden_size))
          return att_f, att_b

In [0]:
class Att_Seq2Seq(Chain):
    def __init__(self, vocab_size, embed_size, hidden_size, batch_size, flag_gpu = True):
        """
        Seq2Seq + Attentionのインスタンス化
        :param vocab_size: 語彙数のサイズ
        :param embed_size: 単語ベクトルのサイズ
        :param hidden_size: 隠れ層のサイズ
        :param batch_size: ミニバッチのサイズ
        :param flag_gpu: GPUを使うかどうか
        """
        super(Att_Seq2Seq, self).__init__(
            # 順向きのEncoder
            f_encoder = LSTM_Encoder(vocab_size, embed_size, hidden_size),
            # 逆向きのEncoder
            b_encoder = LSTM_Encoder(vocab_size, embed_size, hidden_size),
            # Attention Model
            attention = Attention(hidden_size, flag_gpu),
            # Decoder
            decoder = Att_LSTM_Decoder(vocab_size, embed_size, hidden_size)
        )
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        # GPUを使うときはcupy、使わないときはnumpy
        if flag_gpu:
            self.ARR = cuda.cupy
        else:
            self.ARR = np

        # 順向きのEncoderの中間ベクトル、逆向きのEncoderの中間ベクトルを保存するためのリストを初期化
        self.fs = []
        self.bs = []

    def encode(self, words):
        """
        Encoderの計算
        :param words: 入力で使用する単語記録されたリスト
        :return: 
        """
        # 内部メモリ、中間ベクトルの初期化
        c = Variable(self.ARR.zeros((self.batch_size, self.hidden_size), dtype = 'float32'))
        h = Variable(self.ARR.zeros((self.batch_size, self.hidden_size), dtype = 'float32'))
        # まずは順向きのEncoderの計算
        for w in words:
            c, h = self.f_encoder(w, c, h)
            # 計算された中間ベクトルを記録
            self.fs.append(h)

        # 内部メモリ、中間ベクトルの初期化
        c = Variable(self.ARR.zeros((self.batch_size, self.hidden_size), dtype = 'float32'))
        h = Variable(self.ARR.zeros((self.batch_size, self.hidden_size), dtype = 'float32'))
        # 逆向きのEncoderの計算
        for w in reversed(words):
            c, h = self.b_encoder(w, c, h)
            # 計算された中間ベクトルを記録 
            self.bs.insert(0, h)

        # 内部メモリ、中間ベクトルの初期化
        self.c = Variable(self.ARR.zeros((self.batch_size, self.hidden_size), dtype = 'float32'))
        self.h = Variable(self.ARR.zeros((self.batch_size, self.hidden_size), dtype = 'float32'))

    def decode(self, w):
        """
        Decoderの計算
        :param w: Decoderで入力する単語
        :return: 予測単語
        """
        # Attention Modelを使ってEncoderの中間層の加重平均を計算
        att_f, att_b = self.attention(self.fs, self.bs, self.h)
        # Decoderの中間ベクトル、順向きのAttention、逆向きのAttentionを使って次の中間ベクトル、内部メモリ、予測単語の計算    
        t, self.c, self.h = self.decoder(w, self.c, self.h, att_f, att_b)
        return t

    def reset(self):
        """
        インスタンス変数を初期化する
        :return: 
        """
        # 内部メモリ、中間ベクトルの初期化
        self.c = Variable(self.ARR.zeros((self.batch_size, self.hidden_size), dtype = 'float32'))
        self.h = Variable(self.ARR.zeros((self.batch_size, self.hidden_size), dtype = 'float32'))
        # Encoderの中間ベクトルを記録するリストの初期化
        self.fs = []
        self.bs = []
        # 勾配の初期化
        self.cleargrads()

In [0]:
def forward(enc_words, dec_words, model, ARR):
    batch_size = len(enc_words[0])
    model.reset()
    enc_words = [Variable(ARR.array(row, dtype='int32')) for row in enc_words]
    model.encode(enc_words)
    # 損失の初期化
    loss = Variable(ARR.zeros((), dtype='float32'))
    # 精度の初期化
    acc = Variable(ARR.zeros((), dtype='float32'))
    # <eos>をデコーダーに読み込ませる ②
    t = Variable(ARR.array([0 for _ in range(batch_size)], dtype='int32'))
    # デコーダーの計算
    for w in dec_words:
        # 1単語ずつをデコードする ③
        y = model.decode(t)
        # 正解単語をVariable型に変換
        t = Variable(ARR.array(w, dtype='int32'))
        # 正解単語と予測単語を照らし合わせて損失を計算 ④
        loss += functions.softmax_cross_entropy(y, t)
        acc += functions.accuracy(y, t)
    return loss, acc

In [0]:
# trainの関数
def make_minibatch(minibatch):
    # enc_wordsの作成
    enc_words = [row[0] for row in minibatch]
    enc_max = np.max([len(row) for row in enc_words])
    enc_words = np.array([[-1]*(enc_max - len(row)) + row for row in enc_words], dtype='int32')
    enc_words = enc_words.T
    # dec_wordsの作成
    dec_words = [row[1] for row in minibatch]
    dec_max = np.max([len(row) for row in dec_words])
    dec_words = np.array([row + [-1]*(dec_max - len(row)) for row in dec_words], dtype='int32')
    dec_words = dec_words.T
    return enc_words, dec_words

In [0]:
cd marusen575_not_reversed

/content/drive/My Drive/俳句&川柳データセット/marusen575/marusen575_not_reversed


In [0]:
def train(data):
    # 語彙数の確認
    # モデルのインスタンス化
    model = Att_Seq2Seq(
                       vocab_size=vocab_size,
                       embed_size=EMBED_SIZE,
                       hidden_size=HIDDEN_SIZE,
                       batch_size=BATCH_SIZE,
                       flag_gpu=FLAG_GPU)
    # モデルの初期化
    model.reset()
    
    # GPUを使うかどうか決める
    if FLAG_GPU:
        ARR = cupy
        # モデルをGPUのメモリに入れる
        #cuda.get_device(0).use()
        model.to_gpu(0)
    else:
        ARR = np
    loss_list = []
    acc_list = []
    # 学習開始
    for epoch in range(EPOCH_NUM):
        # エポックごとの処理時間計測のためにtime.time()を取得
        start = time.time()
        # エポックごとにoptimizerの初期化
        opt = optimizers.Adam()
        # モデルをoptimizerにセット
        opt.setup(model)
        # 勾配が大きすぎる場合に調整する
        opt.add_hook(optimizer.GradientClipping(5))

        # データのシャッフル
        xr = random.sample(list(data), len(data))
        for num in range(len(data)//BATCH_SIZE):
                minibatch = data[num*BATCH_SIZE: (num+1)*BATCH_SIZE]
                # print('minibatch\n' + str(minibatch))
                # 読み込み用のデータ作成
                enc_words, dec_words = make_minibatch(minibatch)
                # modelのリセット
                model.reset()
                # 順伝播で損失の計算
                total_loss, acc = forward(
                                         enc_words=enc_words,
                                         dec_words=dec_words,
                                         model=model,
                                         ARR=ARR)
                loss_list.append(total_loss)
                acc_list.append(acc)
                # 学習
                # 誤差逆伝播で勾配の計算
                total_loss.backward()
                # 計算した勾配を使ってネットワークを更新
                opt.update()
                # 記録された勾配を初期化する
                opt.use_cleargrads(use=False)
        elapsed_time = (time.time() - start) / 60.0
        print ('Epoch:{}\ttotal loss:{}\ttotal_accuracy:{}\ttime:{}'.format(epoch+1, total_loss, acc, elapsed_time))
        outputpath = OUTPUT_PATH%(EMBED_SIZE, HIDDEN_SIZE, BATCH_SIZE, epoch+1)    
        # エポックごとにモデルの保存
        serializers.save_npz(outputpath, model)

In [0]:
FLAG_GPU = False
OUTPUT_PATH = 'marusen575_not_reversed_EMBED%s_HIDDEN%s_BATCH%s_EPOCH%s.weights'
vocab_size = len(id_to_word)
EMBED_SIZE = 300
HIDDEN_SIZE = 150
BATCH_SIZE = 128
EPOCH_NUM = 30

1~30エポック

In [0]:
print ('開始: ')

train(x_train)

print ('終了: ')

開始: 


NameError: ignored

31〜50エポック

In [0]:
print ('開始: ')

train(x_train)

print ('終了: ')

In [0]:
def forward_test(enc_words, model, ARR):
    ret = []
    model.reset()
    enc_words = [Variable(ARR.array(row, dtype='int32')) for row in enc_words]
    model.encode(enc_words)
    t = Variable(ARR.array([0], dtype='int32'))
    counter = 0
    while counter < 50:
        y = model.decode(t)
        # 次の単語のindexを返す
        label = y.data.argmax()
        ret.append(label)
        t = Variable(ARR.array([label], dtype='int32'))
        counter += 1
        if label == 1:
            counter = 50
    return ret

In [0]:
loaded_model = Att_Seq2Seq(vocab_size,
                       embed_size=EMBED_SIZE,
                       hidden_size=HIDDEN_SIZE,
                       batch_size=BATCH_SIZE,
                       flag_gpu=FLAG_GPU)

In [0]:
serializers.load_npz('marusen575_EMBED300_HIDDEN150_BATCH128_EPOCH60.weights', loaded_model)

In [0]:
def pre(train_or_test_list):
    a = [[x] for x in train_or_test_list]
    a = np.array(a)
    return a

In [0]:
cd ../

/content/drive/My Drive/俳句&川柳データセット/marusen575


In [0]:
id_to_word = pickle_load('id_to_word_marusen575.pickle')
word_to_id = pickle_load('word_to_id_marusen575.pickle')

In [0]:
def id_list_to_senryu(id_list):
    ai_senryu_wordlist = []
    for each_id in id_list:
        ai_senryu_wordlist.append(id_to_word[each_id])
    ai_senryu = ''.join(ai_senryu_wordlist)
    return ai_senryu

def word_to_encode_id(word_list):
    word_list_to_id = []
    for each_word in word_list:
        if each_word not in word_to_id:
            each_word = '<unknown>'
            word_list_to_id.append(word_to_id[each_word])
        else:
            word_list_to_id.append(word_to_id[each_word])
        a = [[x] for x in word_list_to_id]
    b = list(reversed(a))
    b = np.array(b)
    return b

In [0]:
i = 51
encode_id = pre(x_test[i][0])
print(encode_id)
print(id_list_to_senryu(x_test[i][0]))
id_list_to_senryu(list(reversed(x_test[i][0])))

[[   18]
 [ 2664]
 [15280]]
に品所持


'所持品に'

In [0]:
ai_senryu_id = forward_test(encode_id, loaded_model, np)
answer_id = list(x_test[i][1])
print("推論id：" + str(ai_senryu_id))
print("正解id：" + str(answer_id))
del ai_senryu_id[-1]
del answer_id[-1]
print(id_list_to_senryu(ai_senryu_id))
print(id_list_to_senryu(answer_id))

推論id：[23, 1424, 208, 56, 1]
正解id：[425, 41, 2328, 1]
だけどんよ
名前を付ける


#ユーザーの入力フェーズ

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

In [0]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

In [0]:
import re
import numpy as np
import io,sys

In [0]:
import MeCab
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,shell=True).communicate()[0]).decode('utf-8')
m=MeCab.Tagger("-d {0}".format(path))

In [0]:
val = input('一句の5音を入れてください：')
val

一句の5音を入れてください：スマホとね


'スマホとね'

In [0]:
clean_i = re.sub(re.compile('[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％]'), '', val) # 記号の削除
sentence = clean_i.replace("　", "").replace(" ", "") # スペースの消去
node = m.parseToNode(sentence).next #解析してノードに変換

word_list = []
while node.next: #nodeがfalseになるまで
    word_list.append(node.surface)
    node = node.next #次の要素に移動（末尾の場合はfalseになる）
word_list

index = [i for i in word_list]
new_encode_id = word_to_encode_id(index)
print(new_encode_id)

[[   6]
 [1043]]


In [0]:
ai_senryu_id = forward_test(new_encode_id, loaded_model, np)
del ai_senryu_id[-1]
print(id_list_to_senryu(ai_senryu_id))

一度命で<eos>
